# Naïve journey planner

The goal of this notebook running on a local python kernel is to have a naive implementation of a journey planner.
This Naïve journery planner aims at assessing the validity of our transport model at first as well as provide ground predictions for our model. 
In this naïve implementation, our model will consider that public transports always arrive on time, and will try to find a way from point A to reach point B before time T.

In [1]:
# general imports 
import datetime
import pandas as pd
from heapq import heappush, heappop
from collections import deque

pd.set_option("display.max_columns", 50)
import os
import functools
import numpy as np

# for loading the data from spark cluster to local 
from hdfs3 import HDFileSystem

# Import planner implementation
from helpers_naive import *

# for transport modeling 
import networkx as nx

# for visualization 
from helpers_visualisation import *

## Data fetching

First we import the necessary data from the HDFS to model our data 

In [2]:
"""
Loads data from the HDFS to retreive pyspark data wrangling results 
"""
def load_hdfs_to_pandas(filename):
        hdfs = HDFileSystem(host='hdfs://iccluster040.iccluster.epfl.ch', port=8020, user='ebouille') # impersonate ebouille to read the file
        files = hdfs.glob(f'/user/vyuan/final_3/{filename}')
        df = pd.DataFrame()
        for file in files:
            if not 'SUCCESS' in file:
                with hdfs.open(file) as f:
                    df = df.append(pd.read_parquet(f))
        
        
        return df

In [3]:
# get station nodes 
graph_nodes = load_hdfs_to_pandas('graph_nodes.parquet')

#retreive all connections 
trips = load_hdfs_to_pandas("graph_edges_2.parquet")

#check  the sizes to make sure appropriate data
print(f"Length of trips is {len(trips)}")
print(trips.head())


Length of trips is 5192829
       src      dst departure_time arrival_time duration     type
0  8503064  8503065       10:41:00     10:45:00        4  journey
1  8503065  8503074       10:45:00     10:46:00        1  journey
2  8503074  8503068       10:46:00     10:47:00        1  journey
3  8503068  8503066       10:47:00     10:48:00        1  journey
4  8503066  8503075       10:48:00     10:50:00        2  journey


In [4]:
""" 
Data formating for graph modeling
Cast appropriate types of arrival and departure types 
Declare weight and key 
"""
trips["departure_time"] = pd.to_datetime(trips["departure_time"] , format='%H:%M:%S', errors='coerce').dt.time
trips["key"] = trips["arrival_time"].astype(str)
trips["arrival_time"] = pd.to_datetime(trips.arrival_time, format='%H:%M:%S', errors='coerce').dt.time
trips["weight"] = trips["duration"].astype(float)

trips.head()

,src,dst,departure_time,arrival_time,duration,type,key,weight
0,8503064,8503065,10:41:00,10:45:00,4,journey,10:45:00,4.0
1,8503065,8503074,10:45:00,10:46:00,1,journey,10:46:00,1.0
2,8503074,8503068,10:46:00,10:47:00,1,journey,10:47:00,1.0
3,8503068,8503066,10:47:00,10:48:00,1,journey,10:48:00,1.0
4,8503066,8503075,10:48:00,10:50:00,2,journey,10:50:00,2.0


## Graph Creation 

We create the Directed Graph from the edges using networkx, and add an attribute representing the latest arrival for each nodes (stations). 
Each station corresponds to a node and every connection an edge. Edges are thus described by  : 
* Departure time from source 
* Arrival time at destination 
* Type of trip (weither journey or conenction) 
* Weight corresponding to travel time
* Key (unique identifier of edge)

The key attribute alows us to directly instantiate the edge using its arrival time. Such key is unique as the edge also corresponds to given pair of nodes


In [5]:
# computing transport model
master_graph = nx.convert_matrix.from_pandas_edgelist(trips, 
                                                 "src", 
                                                 "dst", 
                                                 edge_key="key",
                                                 edge_attr=["departure_time", "arrival_time", "type","weight"], 
                                                 create_using=nx.MultiDiGraph())

# set the latest arrival time for all nodes at zero 
nx.set_node_attributes(master_graph, datetime.time(0, 0, 0), "latest_arrival")

## Strongly connected component

As we want our journey planner to always be able to work, i.e. to reach a point B from a point A, we need the different stops to be strongly connected.

Thus, we take the maximum strongly connected component, which includes Zurich train station, to restrict our graph to a strongly connected component.

In [6]:
strongly_connected_components = max(nx.strongly_connected_components(master_graph), key=len)

hbb_in_components = '8503000' in strongly_connected_components
print(f"Our strongly connected component has Zurich Train station {hbb_in_components}")

graph_nodes = graph_nodes[graph_nodes["id"].isin(strongly_connected_components)]
graph_strong_connected = master_graph.subgraph(strongly_connected_components)

Our strongly connected component has Zurich Train station True


## Naïve journey planner 

The way we proceeded to levrage our transport model was to use an adapted version of Djikstra's Algorithm to find the shortest path taking into acount the delay between arrival time at a station and next departure time. Yet as our data was made to support all possible trips considering transfers as well, it would have been cumbersome and clumzzy to use it directly on the it. That it why we adopted the following startegy : 

**Backward bfs:** First, we run a Backward bfs starting from target station given a specified arrival time. The idea here was to keep only one edge between to nodes by propagating the latest arrival time to upward nodes. Iteratively, we discarded any nodes having an arrival time being after the latest of the destination node. Yet, one hick up with thi approach is that some edges having departure times from source node earlier in time that it departure time remain. That it is why we had also to perform a forward bfs.

**Dijkstra:** Finally, one the transport model sorted, we implemented a slightly different version of Djikstra algorithm to acount for waiting time. The idea was to adapt the cost function of a given time with such contribution. That way, we were able to output the fastet route from point A to point B given time T. 

## Visualization of your trip 

In order to get a better idea of the actual performance of our model in terms of accuracy and consistency, we decided to build an interactive tool for visualization. This interactive visualization allows to pick departure, arrival stations across all possible ones (allowing for a fully connected transport model) and desired arrival time. The visualization then computes the fastes route and displays the path along the different sations along with the calculated duration in minutes and seconds. It is also to have a glance at the journey details clicking on the toggle "Your trip", to see what when user can leave the starting stations as well as the detailed departures and arrivals for each station. Status of the back-end computations are also shown for the user to know what is going one. 


In [9]:
# declare visualizer 
plot_path(graph_nodes, graph_strong_connected)